# 🔍 Busca Semântica em Documentos PDF com NLP e Transformers

## 📌 Objetivo do Projeto
Este projeto tem como objetivo desenvolver um sistema de **busca semântica**
capaz de recuperar trechos relevantes de documentos textuais a partir do
**significado da consulta do usuário**, indo além da simples correspondência
por palavras-chave.

A solução foi implementada utilizando **modelos de linguagem pré-treinados**
e técnicas modernas de **Processamento de Linguagem Natural (NLP)**, sem o uso
de APIs pagas.

---

## 🧠 O que é Busca Semântica?
Busca semântica é uma abordagem de NLP que representa textos como
**vetores numéricos (embeddings)** em um espaço semântico.

Dessa forma, o sistema consegue identificar trechos **conceitualmente
relacionados** à pergunta do usuário, mesmo quando não há correspondência
literal entre as palavras utilizadas na consulta e no texto original.

---

## 📚 Fonte dos Dados
Como exemplo prático, este projeto utiliza o livro **_Deuses Americanos_**,
de **Neil Gaiman**, fornecido em formato **PDF**.

Entretanto, toda a pipeline foi projetada de forma **genérica**, permitindo
que o mesmo código seja aplicado a **qualquer documento em PDF que contenha
texto extraível**, como:

- Livros  
- Artigos científicos  
- Relatórios  
- Documentação técnica  

O arquivo PDF é convertido para texto bruto antes das etapas de limpeza,
segmentação e processamento semântico.


In [ ]:
# Instalação das bibliotecas necessárias (executar apenas se necessário)

# Extração de texto
# !pip install pdfplumber PyPDF2 ebooklib beautifulsoup4

# Processamento de linguagem natural e embeddings
# !pip install sentence-transformers torch

# Busca vetorial e manipulação de dados
# !pip install faiss-cpu numpy pandas tqdm


In [26]:
# Extração de texto
from PyPDF2 import PdfReader
import pdfplumber

# NLP e embeddings
from sentence_transformers import SentenceTransformer

# Busca vetorial
import faiss

# Manipulação de dados
import numpy as np
import pandas as pd

# Processamento de texto
import re
import string

# Utilidades
import os
from tqdm import tqdm


In [23]:
arquivo = "data/documents/deuses_americanos.epub"
arquivo = "data/documents/deuses_americanos.pdf"

print("Arquivo existe?", os.path.exists(arquivo))
print("Arquivo selecionado:", arquivo)


def testar_pdf(caminho_pdf):
    import pdfplumber
    texto = ""

    with pdfplumber.open(caminho_pdf) as pdf:
        # tenta ler as 5 primeiras páginas
        for i, pagina in enumerate(pdf.pages[:5]):
            pagina_texto = pagina.extract_text()
            if pagina_texto:
                texto += pagina_texto + "\n"

    return texto


def testar_epub(caminho_epub):
    from ebooklib import epub
    from bs4 import BeautifulSoup

    livro = epub.read_epub(caminho_epub)
    textos = []

    for item in livro.get_items():
        # DOCUMENT = conteúdo textual real
        if item.get_type() == item.DOCUMENT:
            soup = BeautifulSoup(item.get_content(), "html.parser")
            texto = soup.get_text(separator=" ", strip=True)
            if len(texto) > 200:  # ignora itens vazios
                textos.append(texto)

    return "\n".join(textos)


ext = os.path.splitext(arquivo)[1].lower()

if ext == ".pdf":
    texto_extraido = testar_pdf(arquivo)

elif ext == ".epub":
    texto_extraido = testar_epub(arquivo)

else:
    raise ValueError("Formato não suportado")


print("\n=== AMOSTRA DO TEXTO EXTRAÍDO ===\n")

if texto_extraido.strip():
    print(texto_extraido[:1500])
else:
    print("❌ Nenhum texto legível encontrado")


Arquivo existe? True
Arquivo selecionado: data/documents/deuses_americanos.pdf

=== AMOSTRA DO TEXTO EXTRAÍDO ===

Copyright © 2011 by Neil Gaiman.
Copyright da edição original © 2001 by Neil Gaiman.
Todos os esforços foram empenhados para localizar e notificar os detentores dos direitos dos materiais
reproduzidos neste livro. Quaisquer omissões que forem identificadas serão corrigidas em edições
posteriores. Agradecemos a permissão para usar os seguintes materiais neste livro:
Trecho de “The Witch of Coos”, de “Two Witches”, em The Poetry of Robert Frost, editado por Edward
Connery Lathem. © 1951 by Robert Frost, © 1923, 1969 by Henry Holt and Co. Reproduzido com
permissão de Henry Holt and Company, LLC.
“Tango Till They’re Sore”, de Tom Waits. Copyright © 1985 by JALMA Music. Usado com permissão.
Todos os direitos reservados.
“Old Friends”, melodia e letra de Stephen Sondheim. Copyright © 1981 Rilting Music, Inc. Todos os
direitos reservados. Usado com permissão. Warner Bros. Publica

In [ ]:
# Caminho do PDF — Deuses Americanos
pdf_path = "data/documents/deuses_americanos.pdf"

texto_completo = ""

with pdfplumber.open(pdf_path) as pdf:
    for i, pagina in enumerate(pdf.pages):
        texto_pagina = pagina.extract_text()
        if texto_pagina:  # ignora páginas sem texto
            texto_completo += texto_pagina + "\n"

# Verificação inicial
print("Total de caracteres extraídos:", len(texto_completo))
print("\n=== AMOSTRA DO TEXTO EXTRAÍDO (DEUSES AMERICANOS) ===\n")
print(texto_completo[:1500])


Total de caracteres extraídos: 1125453

=== AMOSTRA DO TEXTO EXTRAÍDO (DEUSES AMERICANOS) ===

Copyright © 2011 by Neil Gaiman.
Copyright da edição original © 2001 by Neil Gaiman.
Todos os esforços foram empenhados para localizar e notificar os detentores dos direitos dos materiais
reproduzidos neste livro. Quaisquer omissões que forem identificadas serão corrigidas em edições
posteriores. Agradecemos a permissão para usar os seguintes materiais neste livro:
Trecho de “The Witch of Coos”, de “Two Witches”, em The Poetry of Robert Frost, editado por Edward
Connery Lathem. © 1951 by Robert Frost, © 1923, 1969 by Henry Holt and Co. Reproduzido com
permissão de Henry Holt and Company, LLC.
“Tango Till They’re Sore”, de Tom Waits. Copyright © 1985 by JALMA Music. Usado com permissão.
Todos os direitos reservados.
“Old Friends”, melodia e letra de Stephen Sondheim. Copyright © 1981 Rilting Music, Inc. Todos os
direitos reservados. Usado com permissão. Warner Bros. Publications U.S. Inc., Mia

In [24]:
def limpar_texto(texto: str) -> str:
    """
    Limpa e normaliza o texto extraído do PDF.
    - Remove espaços excessivos
    - Remove caracteres estranhos
    - Normaliza quebras de linha
    """

    # Remove caracteres estranhos comuns em PDFs
    texto = re.sub(r'\x0c', ' ', texto)

    # Remove múltiplos espaços
    texto = re.sub(r' +', ' ', texto)

    # Normaliza múltiplas quebras de linha
    texto = re.sub(r'\n{2,}', '\n\n', texto)

    # Remove espaços no início/fim
    texto = texto.strip()

    return texto


# Aplicando limpeza
texto_limpo = limpar_texto(texto_completo)

# Verificação
print("Total de caracteres após limpeza:", len(texto_limpo))
print("\n=== AMOSTRA DO TEXTO LIMPO ===\n")
print(texto_limpo[:1500])


Total de caracteres após limpeza: 1125452

=== AMOSTRA DO TEXTO LIMPO ===

Copyright © 2011 by Neil Gaiman.
Copyright da edição original © 2001 by Neil Gaiman.
Todos os esforços foram empenhados para localizar e notificar os detentores dos direitos dos materiais
reproduzidos neste livro. Quaisquer omissões que forem identificadas serão corrigidas em edições
posteriores. Agradecemos a permissão para usar os seguintes materiais neste livro:
Trecho de “The Witch of Coos”, de “Two Witches”, em The Poetry of Robert Frost, editado por Edward
Connery Lathem. © 1951 by Robert Frost, © 1923, 1969 by Henry Holt and Co. Reproduzido com
permissão de Henry Holt and Company, LLC.
“Tango Till They’re Sore”, de Tom Waits. Copyright © 1985 by JALMA Music. Usado com permissão.
Todos os direitos reservados.
“Old Friends”, melodia e letra de Stephen Sondheim. Copyright © 1981 Rilting Music, Inc. Todos os
direitos reservados. Usado com permissão. Warner Bros. Publications U.S. Inc., Miami, FL 33014.
“In th

In [9]:
def criar_chunks(
    texto: str,
    tamanho_chunk: int = 1000,
    sobreposicao: int = 200
) -> list:
    """
    Divide o texto em blocos (chunks) com sobreposição.
    
    Args:
        texto (str): texto limpo
        tamanho_chunk (int): tamanho máximo de cada bloco
        sobreposicao (int): caracteres reaproveitados entre blocos
    
    Returns:
        list: lista de chunks
    """

    chunks = []
    inicio = 0
    tamanho_texto = len(texto)

    while inicio < tamanho_texto:
        fim = inicio + tamanho_chunk
        chunk = texto[inicio:fim]
        chunks.append(chunk)
        inicio = fim - sobreposicao

    return chunks


# Criando os chunks
chunks = criar_chunks(texto_limpo)

# Verificações
print("Total de chunks criados:", len(chunks))
print("\n=== AMOSTRA DO PRIMEIRO CHUNK ===\n")
print(chunks[0][:1000])


Total de chunks criados: 1407

=== AMOSTRA DO PRIMEIRO CHUNK ===

Copyright © 2011 by Neil Gaiman.
Copyright da edição original © 2001 by Neil Gaiman.
Todos os esforços foram empenhados para localizar e notificar os detentores dos direitos dos materiais
reproduzidos neste livro. Quaisquer omissões que forem identificadas serão corrigidas em edições
posteriores. Agradecemos a permissão para usar os seguintes materiais neste livro:
Trecho de “The Witch of Coos”, de “Two Witches”, em The Poetry of Robert Frost, editado por Edward
Connery Lathem. © 1951 by Robert Frost, © 1923, 1969 by Henry Holt and Co. Reproduzido com
permissão de Henry Holt and Company, LLC.
“Tango Till They’re Sore”, de Tom Waits. Copyright © 1985 by JALMA Music. Usado com permissão.
Todos os direitos reservados.
“Old Friends”, melodia e letra de Stephen Sondheim. Copyright © 1981 Rilting Music, Inc. Todos os
direitos reservados. Usado com permissão. Warner Bros. Publications U.S. Inc., Miami, FL 33014.
“In the Dark wi

In [25]:
# Carregando o modelo de embeddings
modelo = SentenceTransformer("all-MiniLM-L6-v2")

print("Modelo carregado com sucesso!")

# Gerando embeddings para todos os chunks
embeddings = modelo.encode(
    chunks,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

# Verificações
print("\nTotal de embeddings gerados:", embeddings.shape[0])
print("Dimensão de cada embedding:", embeddings.shape[1])

# Amostra de um embedding
print("\n=== AMOSTRA DE UM EMBEDDING ===")
print(embeddings[0][:10])  # primeiros valores


Modelo carregado com sucesso!


Batches:   0%|          | 0/44 [00:00<?, ?it/s]


Total de embeddings gerados: 1407
Dimensão de cada embedding: 384

=== AMOSTRA DE UM EMBEDDING ===
[ 0.02834556 -0.02410486 -0.02806319  0.00534707 -0.0767917   0.03249393
  0.00546049 -0.05791837  0.02381194  0.00687961]


In [27]:
# Dimensão dos embeddings (384 para all-MiniLM-L6-v2)
dimensao = embeddings.shape[1]

# Criação do índice FAISS usando similaridade por cosseno
# Como os embeddings já estão normalizados, usamos Inner Product
index = faiss.IndexFlatIP(dimensao)

# Adicionando os embeddings ao índice
index.add(embeddings)

# Verificações
print("Índice FAISS criado com sucesso!")
print("Total de vetores indexados:", index.ntotal)
print("Dimensão do índice:", dimensao)


Índice FAISS criado com sucesso!
Total de vetores indexados: 1407
Dimensão do índice: 384


In [15]:
def busca_semantica(
    pergunta: str,
    modelo,
    index,
    chunks,
    top_k: int = 5
):
    """
    Realiza busca semântica em um conjunto de textos usando FAISS.

    Parâmetros:
    - pergunta (str): consulta do usuário
    - modelo: modelo SentenceTransformer
    - index: índice FAISS
    - chunks (list): lista de trechos do livro
    - top_k (int): número de resultados retornados

    Retorno:
    - lista de dicionários com score e texto
    """

    # Gerando embedding da pergunta
    embedding_pergunta = modelo.encode(
        [pergunta],
        convert_to_numpy=True,
        normalize_embeddings=True
    )

    # Busca no índice FAISS
    scores, indices = index.search(embedding_pergunta, top_k)

    resultados = []

    for score, idx in zip(scores[0], indices[0]):
        resultados.append({
            "score": float(score),
            "texto": chunks[idx]
        })

    return resultados


In [31]:
pergunta = "O que são os deuses antigos?"

resultados = busca_semantica(
    pergunta=pergunta,
    modelo=modelo,
    index=index,
    chunks=chunks,
    top_k=5
)

for i, r in enumerate(resultados, 1):
    print(f"\nResultado {i}")
    print("Similaridade:", round(r["score"], 4))
    print(r["texto"][:400], "...")



Resultado 1
Similaridade: 0.6284
elhos cavalos de carrossel
antigo pendurados, centenas, alguns precisando de uma demão de tinta, outros de
uma boa espanada. Do teto pendiam dezenas de anjos alados nitidamente feitos
de manequins femininos, alguns com os seios lisos expostos, outros que tinham
perdido a peruca e fitavam a escuridão abaixo, cegos e carecas.
E lá estava o Carrossel.
Uma placa proclamava que era o maior do mundo, an ...

Resultado 2
Similaridade: 0.6136
o copo de suco.
— Madame Rumores disse que você tem falado com todo tipo de gente,
oferecendo todo tipo de coisa. Disse que você está levando o pessoal das antigas
para a guerra — comentou John Chapman.
Shadow e Whiskey Jack estavam arrumando a louça e passando as sobras do
guisado para potes de plástico. Whiskey Jack os guardou sob os montes de neve
do lado de fora, colocando por cima um engradad ...

Resultado 3
Similaridade: 0.6086
das de um centavo, nove sementes de algodão e pelos
de um porco preto —, e mostrou como 